In [ ]:
def get_book_recommendations(query, num_recommendations=5):
    # Perform similarity search
    docs = db_books.similarity_search(query, k=num_recommendations)

    # Get books details for the recommendations
    recommended_books = []
    for doc in docs:
        isbn = doc.page_content.split()[0].strip().replace('"', '')
        book = books[books["isbn13"] == int(isbn)]
        if not book.empty:
            recommended_books.append({
                'title': book['title'].iloc[0],
                'authors': book['authors'].iloc[0],
                'description': book['description'].iloc[0],
                'rating': book['average_rating'].iloc[0]
            })

    return recommended_books


# Test the function with a sample query
recommendations = get_book_recommendations("Science fiction books about space exploration")
for i, book in enumerate(recommendations, 1):
    print(f"\nRecommendation {i}:")
    print(f"Title: {book['title']}")
    print(f"Authors: {book['authors']}")
    print(f"Rating: {book['rating']}")
    print(f"Description: {book['description'][:200]}...")
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from seaborn.external.docscrape import header

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [ ]:
books

In [ ]:
books["tagged_description"]

In [ ]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = ",",
                                   index = False,
                                   header = False)

In [ ]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [ ]:
documents[0]

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can proceed with your original code
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

In [104]:
# Remove quotes and convert to integer
isbn = docs[0].page_content.split()[0].strip().replace('"', '')
books[books["isbn13"] == int(isbn)]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [105]:
print(docs[0].page_content)


"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."
